### Importing Librararies

In [1]:
import pandas as pd
import numpy as np

from functions import *

from tqdm import tqdm_notebook as tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

import re
from bs4 import BeautifulSoup

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



[nltk_data] Downloading package wordnet to /Users/leo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/leo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/leo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Data Exploration and Preprocessing

In [2]:
#Opening and storing train data
train_data = open('data/training_set.txt', "r")
train_data = train_data.readlines()

# remove /n at the end of each line
for index, line in enumerate(train_data):
      train_data[index] = line.strip()

#Opening and storing dev data
dev_data = open('data/dev_set.txt', "r")
dev_data = dev_data.readlines()

# remove /n at the end of each line
for index, line in enumerate(dev_data):
      dev_data[index] = line.strip()

train_data[:5], dev_data[:5]
#It seems that every line is a different training sentence followed by its respective emotion

(['sentence\temotion',
  "I'm too old to be traded in .\t6",
  'Mother said you could always tell a lady by her hands .\t8',
  "I always said I'd leave off when the time came .\t6",
  "He'll be safe with me .\t2"],
 ['sentence\temotion',
  'What happens to the gold in our safe ?\t4',
  'Natural to get cold feet .\t8',
  'Not very lucky , is he ?\t7',
  "I'm just a little anxious to get up there and whoop ET's ass , that's all .\t2"])

In [3]:
train_df = txt_to_df(train_data)
dev_df = txt_to_df(dev_data)
train_df.head()

,sentence,emotion
1,I'm too old to be traded in .,6
2,Mother said you could always tell a lady by he...,8
3,I always said I'd leave off when the time came .,6
4,He'll be safe with me .,2
5,Lay off .,1


In [4]:
dev_df.head()

,sentence,emotion
1,What happens to the gold in our safe ?,4
2,Natural to get cold feet .,8
3,"Not very lucky , is he ?",7
4,I'm just a little anxious to get up there and ...,2
5,Did you think we don't know about your affair ...,1


In [5]:
#Visualising the distributions of different emotions

label_counter(train_df,"emotion", set_name= "Training"), label_counter(dev_df, "emotion", "Dev")

(None, None)

In [6]:
word_counter(train_df["sentence"], "Training"), word_counter(dev_df["sentence"], "Dev")

(None, None)

In [10]:
clean(list(train_df["sentence"]), True, True)

  0%|          | 0/14000 [00:00<?, ?it/s]

/Users/leo/Desktop/nova/text_mining/other/functions.py:81: GuessedAtParserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 81 of the file /Users/leo/Desktop/nova/text_mining/other/functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.




['i m too old to be trade in',
 'mother said you could alway tell a ladi by her hand',
 'i alway said i d leav off when the time came',
 'he ll be safe with me',
 'lay off',
 'you tell him to take care of you',
 'i hope so',
 'whi do you want to shut me out in the cold like this',
 'you taught me a lesson man',
 'i ll do everyth i can to make a success of it',
 'but it wa worth it',
 'i hope i never wake up',
 'alright here s how it work',
 'flo be a parent is about share',
 'he look like a drunk',
 'primit',
 'hello never ever pick up the phone',
 'person could i have a word with you',
 'i m not resign my senior partnership the day after what happen happen if that s what you re ask',
 'it s ok',
 'whi don t you sit down and tri not to wast my time',
 'find someon to type this',
 'you re too nice',
 'it s time to updat the program and i would like to hear from each one of you',
 'i ve lost anoth man and that bastard think it s funni',
 'i hope you know that',
 'don t you like child',
 